<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_8_Second3Dproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# ============================================================
# 3D Truss FEM (Problem 8) - MATLAB -> Python (Google Colab)
# copy & run in 1 cell
# ============================================================

def solution(GDof, prescribedDof, K, F, u_prescribed=None):
    if u_prescribed is None:
        u_prescribed = np.zeros(len(prescribedDof), dtype=float)

    prescribed = np.array([d-1 for d in prescribedDof], dtype=int)  # 0-based
    all_dofs = np.arange(GDof, dtype=int)
    free = np.setdiff1d(all_dofs, prescribed)

    U = np.zeros(GDof, dtype=float)
    U[prescribed] = u_prescribed

    K_ff = K[np.ix_(free, free)]
    K_fp = K[np.ix_(free, prescribed)]
    F_f  = F[free]

    rhs = F_f - K_fp @ U[prescribed]
    U[free] = np.linalg.solve(K_ff, rhs)
    return U


def outputDisplacementsReactions(U, K, GDof, prescribedDof, F):
    R = K @ U - F
    print("=== DISPLACEMENTS (U) ===")
    for i in range(GDof):
        print(f"DOF {i+1}: U = {U[i]:.6e}")

    print("\n=== REACTIONS AT PRESCRIBED DOFs ===")
    for d in prescribedDof:
        print(f"DOF {d}: R = {R[d-1]:.6e}")


def formStiffness3Dtruss(GDof, numberElements, elementNodes, nodeCoordinates, E, A):
    K = np.zeros((GDof, GDof), dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]  # 1-based

        x1, y1, z1 = nodeCoordinates[n1-1, :]
        x2, y2, z2 = nodeCoordinates[n2-1, :]

        dx = x2 - x1
        dy = y2 - y1
        dz = z2 - z1
        L = np.sqrt(dx*dx + dy*dy + dz*dz)

        l = dx / L
        m = dy / L
        n = dz / L

        k = (E * A[e] / L) * np.array([
            [ l*l,  l*m,  l*n, -l*l, -l*m, -l*n],
            [ l*m,  m*m,  m*n, -l*m, -m*m, -m*n],
            [ l*n,  m*n,  n*n, -l*n, -m*n, -n*n],
            [-l*l, -l*m, -l*n,  l*l,  l*m,  l*n],
            [-l*m, -m*m, -m*n,  l*m,  m*m,  m*n],
            [-l*n, -m*n, -n*n,  l*n,  m*n,  n*n],
        ], dtype=float)

        edof = np.array([
            3*n1-2, 3*n1-1, 3*n1,
            3*n2-2, 3*n2-1, 3*n2
        ], dtype=int) - 1  # 0-based

        K[np.ix_(edof, edof)] += k

    return K


def stresses3Dtruss(numberElements, elementNodes, nodeCoordinates, U, E):
    sigma = np.zeros(numberElements, dtype=float)

    for e in range(numberElements):
        n1, n2 = elementNodes[e, 0], elementNodes[e, 1]

        x1, y1, z1 = nodeCoordinates[n1-1, :]
        x2, y2, z2 = nodeCoordinates[n2-1, :]

        dx = x2 - x1
        dy = y2 - y1
        dz = z2 - z1
        L = np.sqrt(dx*dx + dy*dy + dz*dz)

        l = dx / L
        m = dy / L
        n = dz / L

        edof = np.array([
            3*n1-2, 3*n1-1, 3*n1,
            3*n2-2, 3*n2-1, 3*n2
        ], dtype=int) - 1

        u_e = U[edof]
        sigma[e] = (E / L) * np.array([-l, -m, -n, l, m, n], dtype=float) @ u_e

    print("\nstresses")
    print(sigma)
    return sigma


# ============================================================
# Main (equivalent to problem8.m)
# ============================================================

E = 210000.0
A = np.array([100.0, 100.0, 100.0, 100.0], dtype=float)

nodeCoordinates = np.array([
    [4000.0,  4000.0, 3000.0],
    [0.0,     4000.0,    0.0],
    [0.0,     4000.0, 6000.0],
    [4000.0,     0.0, 3000.0],
    [8000.0, -1000.0, 1000.0],
], dtype=float)

elementNodes = np.array([
    [1, 2],
    [1, 3],
    [1, 4],
    [1, 5],
], dtype=int)

numberElements = elementNodes.shape[0]
numberNodes = nodeCoordinates.shape[0]
GDof = 3 * numberNodes

U = np.zeros(GDof, dtype=float)
F = np.zeros(GDof, dtype=float)

# MATLAB: force(2)=-10000  => DOF2 = uy of node 1
F[1] = -10000.0

# Stiffness
K = formStiffness3Dtruss(GDof, numberElements, elementNodes, nodeCoordinates, E, A)

# MATLAB: prescribedDof = [4:15]'
prescribedDof = list(range(4, 16))

# Solve
U = solution(GDof, prescribedDof, K, F)

# Output
outputDisplacementsReactions(U, K, GDof, prescribedDof, F)

# Element stresses
sigma = stresses3Dtruss(numberElements, elementNodes, nodeCoordinates, U, E)


=== DISPLACEMENTS (U) ===
DOF 1: U = -3.023681e-01
DOF 2: U = -1.517731e+00
DOF 3: U = 2.687716e-01
DOF 4: U = 0.000000e+00
DOF 5: U = 0.000000e+00
DOF 6: U = 0.000000e+00
DOF 7: U = 0.000000e+00
DOF 8: U = 0.000000e+00
DOF 9: U = 0.000000e+00
DOF 10: U = 0.000000e+00
DOF 11: U = 0.000000e+00
DOF 12: U = 0.000000e+00
DOF 13: U = 0.000000e+00
DOF 14: U = 0.000000e+00
DOF 15: U = 0.000000e+00

=== REACTIONS AT PRESCRIBED DOFs ===
DOF 4: R = 2.709218e+02
DOF 5: R = 0.000000e+00
DOF 6: R = 2.031913e+02
DOF 7: R = 1.354609e+03
DOF 8: R = 0.000000e+00
DOF 9: R = -1.015957e+03
DOF 10: R = 0.000000e+00
DOF 11: R = 7.968087e+03
DOF 12: R = 0.000000e+00
DOF 13: R = -1.625531e+03
DOF 14: R = 2.031913e+03
DOF 15: R = 8.127654e+02

stresses
[ -3.38652236 -16.9326118  -79.68086584 -27.26097914]
